In [1]:
!pip install -q llama-index openai PyMuPDF pandas
!pip install -q tqdm
!pip install -q transformers
!pip install -q nltk

In [2]:
import fitz  # PyMuPDF
import pandas as pd
import os
import random
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import time
import csv
import torch
import nltk
nltk.download('punkt')  
import re
import unicodedata
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\engba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\engba\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
txt_folder = "C:/Users/engba/Desktop/New folder"
files = os.listdir(txt_folder)

In [4]:
# Keywords to process the text
keywords = ["ACKNOWLEDGMENTS", "References", "Acknowledgments", "REFERENCES"]

# Loop through the files for processing
for file in os.listdir(txt_folder):
    if file.endswith(".txt"):
        file_path = os.path.join(txt_folder, file)
        print(f"--- {file} ---")
        
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            
            # Process the text based on keywords
            last_index = -1
            for keyword in keywords:
                index = text.rfind(keyword)  # Get the index of the last occurrence of the keyword
                if index > last_index:
                    last_index = index
            
            if last_index != -1:
                text = text[:last_index]
            
            # Print or perform further operations on processed text
            print(text)  # For example, print the processed text to the console


--- 0704.0548v1.txt ---
 
Neutrinos & Non-proliferation in Europe 
Michel Cribier*  
APC, Paris 
CEA/Saclay, DAPNIA/SPP 
The International Atomic Energy Agency (IAEA) is the United Nations agency in 
charge of the development of peaceful use of atomic energy. In particular IAEA is the 
verification authority of the Treaty on the Non-Proliferation of Nuclear Weapons (NPT). To 
do that jobs inspections of civil nuclear installations and related facilities under safeguards 
agreements are made in more than 140 states. 
IAEA uses many different tools for these verifications, like neutron monitor, gamma 
spectroscopy, but also bookeeping of the isotopic composition at the fuel element level before 
and after their use in the nuclear power station. In particular it verifie that weapon-origin and 
other fissile materials that Russia and USA have released from their defense programmes are 
used for civil application.  
The existence of an antineutrino signal sensitive to the power and to the i

In [5]:
def split_into_sentences(text):
    sentences = sent_tokenize(text)
    return sentences

# Dividing the text into sentences
sentences = split_into_sentences(text)

# Print each sentence on the screen
for i, sentence in enumerate(sentences):
    print(f"Sentence {i + 1}: {sentence}")

Sentence 1:  
Neutrinos & Non-proliferation in Europe 
Michel Cribier*  
APC, Paris 
CEA/Saclay, DAPNIA/SPP 
The International Atomic Energy Agency (IAEA) is the United Nations agency in 
charge of the development of peaceful use of atomic energy.
Sentence 2: In particular IAEA is the 
verification authority of the Treaty on the Non-Proliferation of Nuclear Weapons (NPT).
Sentence 3: To 
do that jobs inspections of civil nuclear installations and related facilities under safeguards 
agreements are made in more than 140 states.
Sentence 4: IAEA uses many different tools for these verifications, like neutron monitor, gamma 
spectroscopy, but also bookeeping of the isotopic composition at the fuel element level before 
and after their use in the nuclear power station.
Sentence 5: In particular it verifie that weapon-origin and 
other fissile materials that Russia and USA have released from their defense programmes are 
used for civil application.
Sentence 6: The existence of an antineutri

In [8]:
def split_into_sentences(text):
    sentences = sent_tokenize(text)
    return sentences

In [6]:
# Define a function to create paragraphs from text
def create_paragraphs(text, max_chars=500, max_sentences=5):
    # Split text into sentences
    sentences = split_into_sentences(text)
    paragraphs = []  # Initialize an empty list to store paragraphs
    current_paragraph = ""  # Initialize an empty string to accumulate sentences into paragraphs

    for sentence in sentences:
        # Check if adding the current sentence exceeds the maximum characters or sentences limit
        if len(current_paragraph) + len(sentence) < max_chars and len(current_paragraph.split('.')) <= max_sentences:
            current_paragraph += sentence.strip() + " "  # Add sentence to current paragraph
        else:
            paragraphs.append(current_paragraph.strip())  # Add completed paragraph to list
            current_paragraph = sentence.strip() + " "  # Start a new paragraph with the current sentence

    # Add the last remaining paragraph
    if current_paragraph:
        paragraphs.append(current_paragraph.strip())

    return paragraphs  # Return the list of paragraphs


In [7]:
# Split the text into paragraphs
paragraphs = create_paragraphs(text)

# Print each paragraph and its sentences
for i, paragraph in enumerate(paragraphs):
    # Print the heading for each paragraph
    print(f"Context {i + 1}:")
    
    # Split paragraph into sentences
    sentences = split_into_sentences(paragraph)
    
    # Join sentences into a single string with each sentence stripped of extra whitespace
    paragraph_text = " ".join(sentence.strip() for sentence in sentences)
    print(paragraph_text)
    
    # Print an empty line between paragraphs, except for the last one
    if i < len(paragraphs) - 1:
        print()


Context 1:
Neutrinos & Non-proliferation in Europe 
Michel Cribier*  
APC, Paris 
CEA/Saclay, DAPNIA/SPP 
The International Atomic Energy Agency (IAEA) is the United Nations agency in 
charge of the development of peaceful use of atomic energy. In particular IAEA is the 
verification authority of the Treaty on the Non-Proliferation of Nuclear Weapons (NPT). To 
do that jobs inspections of civil nuclear installations and related facilities under safeguards 
agreements are made in more than 140 states.

Context 2:
IAEA uses many different tools for these verifications, like neutron monitor, gamma 
spectroscopy, but also bookeeping of the isotopic composition at the fuel element level before 
and after their use in the nuclear power station. In particular it verifie that weapon-origin and 
other fissile materials that Russia and USA have released from their defense programmes are 
used for civil application.

Context 3:
The existence of an antineutrino signal sensitive to the power and to

In [31]:
# # CSV document creating and writing paragraphs
# csv_file = 'C:/Users/engba/Desktop/context.csv'
# with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)
#     writer.writerow(['context'])  # Başlık satırı yazdırma
    
#     for paragraph in paragraphs:
#         writer.writerow([paragraph])

# print(f"{len(paragraphs)} paragraphs have been written to {csv_file}.")

56 paragraphs have been written to C:/Users/engba/Desktop/context.csv.


# Context Creator

In [11]:
# Define a function to split text into sentences
def split_into_sentences(text):
    sentences = sent_tokenize(text)
    return sentences

In [12]:
# Define a function to create paragraphs
def create_paragraphs(text, max_chars=4096, max_sentences=12):
    sentences = split_into_sentences(text)
    paragraphs = []
    current_paragraph = ""

    for sentence in sentences:
        # Delete text such as [Mik77] and [ and ] characters in it
        cleaned_sentence = re.sub(r'\[.*?\]', '', sentence)
        cleaned_sentence = ' '.join(cleaned_sentence.split())
        
        # If a new sentence is added, it must not exceed the maximum character limit or the number of sentences limit
        if len(current_paragraph) + len(cleaned_sentence) < max_chars and len(current_paragraph.split('.')) <= max_sentences:
            current_paragraph += cleaned_sentence.strip() + " "
        else:
            # Son paragrafta gereksiz karakterleri temizleme
            current_paragraph = re.sub(r'-\s*\d+\s*-', '', current_paragraph)
            current_paragraph = re.sub(r'\s+', ' ', current_paragraph)  # Birden fazla boşluğu tek boşlukla değiştirme
            paragraphs.append(current_paragraph.strip())
            current_paragraph = cleaned_sentence.strip() + " "

    # Add the last remaining paragraph
    if current_paragraph:
        current_paragraph = re.sub(r'-\s*\d+\s*-', '', current_paragraph)
        current_paragraph = re.sub(r'\s+', ' ', current_paragraph)  # Birden fazla boşluğu tek boşlukla değiştirme
        paragraphs.append(current_paragraph.strip())

    return paragraphs

In [13]:
# Divide text into paragraphs
paragraphs = create_paragraphs(text)

# Creating a CSV file and writing paragraphs
csv_file = 'C:/Users/engba/Desktop/context.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['context'])  # Title line printing
    
    for paragraph in paragraphs:
        writer.writerow([paragraph])

print(f"{len(paragraphs)} paragraphs have been written to {csv_file}.")

12 paragraphs have been written to C:/Users/engba/Desktop/context.csv.


In [26]:
# T5 model ve tokenizer
model_name = "ramsrigouthamg/t5_squad_v1"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [27]:
# CSV file path
input_file = r"C:\Users\engba\Desktop\context.csv"
output_file = r"C:\Users\engba\Desktop\questions_context.csv"

In [28]:
df = pd.read_csv(input_file)

# Empty lists to collect questions and paragraphs
questions = []
contexts = []

In [29]:
# Process for each line
for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating Questions"):
    context_paragraph = row["context"]

    # Generator
    input_text = f"generate question: {context_paragraph}"
    if not input_text.endswith("</s>"):
        input_text += " </s>"

    # Create a question
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    output_ids = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True).replace("question:", "").strip()

    # Add to data frame
    questions.append(question)
    contexts.append(context_paragraph)

Generating Questions:   8%|▊         | 1/12 [00:03<00:40,  3.72s/it]c:\Users\engba\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Generating Questions: 100%|██████████| 12/12 [00:35<00:00,  2.95s/it]


In [30]:
# # Create to data frame
output_df = pd.DataFrame({"questions": questions, "context": contexts})

# Save to new csv file
output_df.to_csv(output_file, index=False)

print(f"Questions and context saved to {output_file}")

Questions and context saved to C:\Users\engba\Desktop\questions_context.csv
